In [1]:
# Install CIOOS-Ocean-Standards package
!pip install git+git://github.com/cioos-siooc/cioos-siooc-standards.git@jessy-dev#egg=version_subpkg\&subdirectory=ocean_standards 
 

  Cloning git://github.com/cioos-siooc/cioos-siooc-standards.git (to revision jessy-dev) to /tmp/pip-install-fzoeo5zz/version-subpkg
  Running command git clone -q git://github.com/cioos-siooc/cioos-siooc-standards.git /tmp/pip-install-fzoeo5zz/version-subpkg
  Running command git checkout -b jessy-dev --track origin/jessy-dev
  Switched to a new branch 'jessy-dev'
  Branch 'jessy-dev' set up to track remote branch 'jessy-dev' from 'origin'.
  Created wheel for cioos-standards: filename=cioos_standards-0.1.0-cp37-none-any.whl size=43934 sha256=0cd93719803117ab98329b4a512e8c0779d72d5d50012d05ee88ad83dbca4697
  Stored in directory: /tmp/pip-ephem-wheel-cache-a2rkay9y/wheels/6c/ae/53/c4b40a4509f26f103a75e56ecd62e84de233b79d55ae0e4351
  ERROR: Failed building wheel for cioos-standards
  Running setup.py clean for cioos-standards
  ERROR: Failed cleaning build dir for cioos-standards
Successfully built cioos-standards
Failed to build cioos-standards


In [78]:
# Import modules
from ocean_standards import cf, erddap, nerc
import pandas as pd
import warnings
from tqdm.auto import tqdm
import re
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Import GF3 to vocabulary mapping sheet


In [111]:
vocabulary_table_link = "https://raw.githubusercontent.com/cioos-siooc/cioos-siooc_data_transform/improve-xarray-tools/projects/odf_transform/odf_transform/vocabulary/CIOOS_Vocabulary_List%20-%20Complete_Vocabulary.csv"
df = pd.read_csv(vocabulary_table_link)
original_columns = df.columns

In [7]:
# Retrieve Vocabularies id and prefLabel for P01, P07 and CF standard name 
df_P01 = nerc.get_nvs_variable_info(vocabulary='P01')
df_P06 = nerc.get_nvs_variable_info(vocabulary='P06')
df_P07 = nerc.get_nvs_variable_info(vocabulary='P07')
df_cf = cf.get_cf_names()

# P01 BODC 

In [28]:
# Filter Selected P01
selected_p01_list = df['SDN:P01::id'].dropna().drop_duplicates()
df_P01_full_table = pd.DataFrame()
for nerc_id in tqdm(selected_p01_list):
    try:
        df_P01_full_table = pd.concat((df_P01_full_table, nerc.get_nvs_variable_info(nerc_id=nerc_id)))
    except:
        warnings.warn('Invalir URL: {0}'.format(nerc_id))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Invalir URL: None
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Invalir URL: ToBeDefined
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Invalir URL: http://vocab.nerc.ac.uk/collection/S29/current/ATTNZS01
  


In [52]:
# For each matched terms within GF3 update the columns based on the NERC_ID
for nerc_id, row in df_P01_full_table.set_index('@id').iterrows():
    is_matching_p01 = df['SDN:P01::id'] == nerc_id
    df.loc[is_matching_p01,'SDN:P01::urn'] = re.search("\/([A-Za-z0-9\_]*)\/*$",nerc_id)[1]
    df.loc[is_matching_p01,'SDN:P01::prefLabel'] = row['prefLabel-en']
    df.loc[is_matching_p01,'SDN:P01::def'] = row['prefLabel-en']
    df.loc[is_matching_p01,'definition'] = row['definition-en']

df['sdn_parameter_urn'] = "SDN:P01::"+ df['SDN:P01::urn']
df['sdn_parameter_name'] = df['SDN:P01::prefLabel']
    

# CF Standard Name P07

In [61]:
cf_to_p07 = {row['prefLabel']:row['@id'] for index, row in df_P07.iterrows() }
df['SDN:P07::id'] = df['standard_name'].replace(cf_to_p07)

# P06 Units 

In [81]:
def retrieve_p06(unit):
    if type(unit) == list:
        unit = unit[0]
    try: 
        return nerc.get_p06_units_id(unit)
    except:
        return None

df['suggested_P06_id'] = df['expected_units'].str.split('|').progress_apply(retrieve_p06)
p06_label = df_P06.set_index('@id')['prefLabel'].to_dict()

In [87]:
df['suggested_P06_label'] = df['suggested_P06_id'].replace(df_P06.set_index('@id')['prefLabel'].to_dict())

In [93]:
# If missing term fill it with the suggested term
df['SDN:P06::id'] = df['SDN:P06::id'].fillna(df['suggested_P06_id'])
df['SDN:P06::prefLabel'] = df['SDN:P06::id'].fillna(df['suggested_P06_label'])

In [103]:
# Show conflicting suggestions
conflicted_P06_indexes = df[df['SDN:P06::id']!= df["suggested_P06_id"]].dropna(subset=['SDN:P06::id','suggested_P06_id']).index
df.loc[conflicted_P06_indexes]

,Vocabulary,name,Users,MEDS:STATUS,MEDS:CATEGORY_MEANING_E,MEDS:CATEGORY_MEANING_F,MEDS:UNITS_DESCRIPTOR_E,MEDS:UNITS_DESCRIPTOR_F,MEDS:CF_CODE,MEDS:NETCDF_LONG_NAME,MEDS:WMO_CODE_TABLE_ID,MEDS:CONVENTION,BIO:units,BIO:P06 urn,BIO:P06 name,BIO:P01 urn,BIO:P01 name,BIO:CF_CODE,IML:definition,IML:units,long_name,expected_units,expected_scale,expected_instrument,expected_calibration_review,variable_name,standard_name,SDN:P07::id,SDN:P01::id,SDN:P01::urn,SDN:P01::prefLabel,sdn_parameter_urn,sdn_parameter_name,SDN:P06::id,SDN:P06::urn,SDN:P06::prefLabel,sdn_uom_urn,sdn_uom_name,definition,convention,scale_factor,GF3_name,SDN:P01::Alternative Name,SDN:P01::def,suggested_P06_id,suggested_P06_label
84,GF3,ATTU,"BIO,IML",A,Instrument characteristics,Caractéristiques de l'instrument,per metre,par mètre,NaN,NaN,NaN,NaN,/m,NOPM,number per metre,ATTNXXZZ,Attenuation (unspecified wavelength) per unit ...,NaN,NaN,NaN,NaN,/m|per metre,NaN,NaN,NaN,NaN,NaN,NaN,http://vocab.nerc.ac.uk/collection/P01/current...,ATTNXXZZ,Attenuation (unspecified wavelength) per unit ...,SDN:P01::ATTNXXZZ,Attenuation (unspecified wavelength) per unit ...,http://vocab.nerc.ac.uk/collection/P06/current...,NOPM,http://vocab.nerc.ac.uk/collection/P06/current...,SDN:P06::NOPM,Number per metre,Unavailable,NaN,NaN,ATTU,NaN,Attenuation (unspecified wavelength) per unit ...,http://vocab.nerc.ac.uk/collection/P06/current...,per metre
102,GF3,BEAM,"BIO,IML,MEDS",A,Instrument characteristics,Caractéristiques de l'instrument,dimensionless,sans dimension,NaN,NaN,NaN,NaN,NaN,UCNT,counts,"TNIHCE(01,02,03,04",Echo intensity from the water body by moored a...,NaN,NaN,NaN,NaN,dimensionless,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://vocab.nerc.ac.uk/collection/P06/current...,UCNT,http://vocab.nerc.ac.uk/collection/P06/current...,SDN:P06::UCNT,Counts,NaN,NaN,NaN,BEAM,NaN,NaN,http://vocab.nerc.ac.uk/collection/P06/current...,Dimensionless
410,GF3,LATD,"BIO,IML,MEDS",A,Position and navigation,Position et navigation,degree(angle),degré(angle),NaN,NaN,NaN,NaN,degrees,UAAA,degrees,ALATZZ01,Latitude north,latitude,NaN,NaN,NaN,degrees_north|degree|deg|degrees,NaN,NaN,NaN,NaN,latitude,http://vocab.nerc.ac.uk/collection/P07/current...,http://vocab.nerc.ac.uk/collection/P01/current...,ALATZZ01,Latitude north,SDN:P01::ALATZZ01,Latitude north,http://vocab.nerc.ac.uk/collection/P06/current...,UAAA,http://vocab.nerc.ac.uk/collection/P06/current...,SDN:P06::UAAA,Degrees,Unavailable,NaN,NaN,LATD,NaN,Latitude north,http://vocab.nerc.ac.uk/collection/P06/current...,Degrees north
424,GF3,LOND,"BIO,IML,MEDS",A,Position and navigation,Position et navigation,degree(angle),degré(angle),NaN,NaN,NaN,NaN,degrees,UAAA,degrees,ALONZZ01,Longitude east,longitude,NaN,NaN,NaN,degrees_east|degree|deg|degrees,NaN,NaN,NaN,NaN,longitude,http://vocab.nerc.ac.uk/collection/P07/current...,http://vocab.nerc.ac.uk/collection/P01/current...,ALONZZ01,Longitude east,SDN:P01::ALONZZ01,Longitude east,http://vocab.nerc.ac.uk/collection/P06/current...,UAAA,http://vocab.nerc.ac.uk/collection/P06/current...,SDN:P06::UAAA,Degrees,Unavailable,NaN,NaN,LOND,NaN,Longitude east,http://vocab.nerc.ac.uk/collection/P06/current...,Degrees east
755,GF3,STRD,"BIO,IML",A,Position and navigation,Position et navigation,degree(angle) True,degré(angle) vrai,NaN,NaN,NaN,NaN,degrees,UAAA,degrees,NaN,NaN,NaN,NaN,NaN,NaN,degree True|degree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://vocab.nerc.ac.uk/collection/P06/current...,UAAA,http://vocab.nerc.ac.uk/collection/P06/current...,SDN:P06::UAAA,Degrees,NaN,NaN,NaN,STRD,NaN,NaN,http://vocab.nerc.ac.uk/collection/P06/current...,Degrees True
823,GF3,TRAN,"BIO,IML,MEDS",A,Biological Oceanography,Océanographie biologique,percent per metre,pourcentage par mètre,NaN,NaN,NaN,NaN,%,UPCT,percent,NaN,NaN,NaN,NaN,NaN,NaN,percent per metre|percent|%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://vocab.nerc.ac.uk/collection/P06/current...,UPCT,http://vocab.nerc.ac.uk/collection/P06/current...,SDN:P06::

In [104]:
# Apply suggested terms
df.loc[conflicted_P06_indexes,'SDN:P06::id'] = df.loc[conflicted_P06_indexes,'suggested_P06_id']
df.loc[conflicted_P06_indexes,'SDN:P06::prefLabel'] = df.loc[conflicted_P06_indexes,'suggested_P06_label']


In [110]:
# Apply restul to sdn attributes
df['sdn_uom_urn'] = 'SDN:P06::' + df['SDN:P06::id'].str.extract("\/([A-Za-z0-9\_]*)\/*$")
df['sdn_uom_name'] = df['SDN:P06::prefLabel']

# Ouput corrected Vocabulary

In [112]:
df[original_columns].to_csv('CIOOS_Vocabulary_List - Complete_Vocabulary.csv')